# **Solar API로 번역하기**
> **SamSum 데이터셋의 라이센스는 cc-by-nc-nd-4.0로 변경 금지 데이터임을 뒤늦게 인지하여 수정하였습니다. 변경 가능한 데이터셋을 이용해주세요.**  
> **해당 데이터를 학습 데이터 활용하여 오히려 점수가 떨어진 점 참고바랍니다.**  
> ~~아래 코드는 **Upstage AI Lab**의 일상 대화 요약 대회에서 **Solar API를 활용**해 **[SamSum](https://huggingface.co/datasets/samsum)** 데이터셋을 번역했던 코드입니다.~~   
> ~~**DialogSum** 데이터셋과 유사한 **SamSum** 데이터셋을 번역하였으며 **DialogSum 데이터셋과 최대한 비슷해 지도록** 전처리 과정을 포함하고있습니다.~~

In [ ]:
import pandas as pd
import os
import json
import glob

import random
import re

## 1 Data Load

In [ ]:
folder_path = "../data"

with open(os.path.join(folder_path, "train.json"), "r", encoding='UTF8') as f:
    train_data = json.load(f)

In [ ]:
df_train = pd.DataFrame(train_data)
df_train = df_train[df_train['dialogue'] != ""].reset_index(drop=True)

del train_data

In [ ]:
df_train

## 2 Data Process

In [ ]:
# 정규식 패턴에 해당하는 문자열을 replace하는 함수
def remove_extra_spc(x, pattern, replace_text):
    return re.sub(pattern, replace_text, x)

In [ ]:
# 이모지 및 특수문자 확인
pattern = r"[^a-zA-Z0-9\s!$%&*_+-=~'\"\|:\\.,/?]"
sp_list = []
for data in df_train['dialogue']:
    sp_list += re.findall(pattern, data)
    
    sp_list = list(set(sp_list))
    
sp_list, len(sp_list)

In [ ]:
# 개행문자 기준으로 대화를 나눴을 때 비어있는 리스트 제거
def remove_empty(x):
    text_list = x.copy()
    for i in range(len(x)):
        if x[i] == "":
            print("empty!")
            text_list.pop(i)
            
    return text_list

In [ ]:
# 마침표가 없는 문장에 마침표 추가
def add_fullstop(x):
    pattern = r"[^!?.]$"
    match = re.search(pattern, x)
    if match:
        x+="."
    return x

In [ ]:
# 한사람이 연속적으로 발화하는 경우 해당 문장을 합쳐서 반환(원본 학습데이터와 비슷하게 맞추기 위함)
def continuous_talking_paltten(data):
    speak_list = []
    stack_text = data[0]

    pattern = '[a-zA-Z\'\s\-,\._]+:'
    match_now = re.match(pattern, data[0])
    # now_person = match_now.group(0)
    
    try:
        now_person = match_now.group(0)
    except Exception as e:
        # print(e)
        # print(data)
        stack_text += "#Error_Text#"

    for i in range(1, len(data)):    
        match_now = re.match(pattern, data[i])
        
        try:
            if now_person == match_now.group(0):
                now_text = re.sub(pattern, "", data[i], count=1)
                stack_text = stack_text + " " + now_text
                
                if i == len(data)-1:
                    speak_list.append(stack_text)
            else:
                now_person = match_now.group(0)
                speak_list.append(stack_text)
                stack_text = data[i]
                
                if i == len(data)-1:
                    speak_list.append(stack_text)
        except Exception as e:
            # print(e)
            # print(data)
            speak_list.append("#Error_Text#" + data[i])
            
    
    return speak_list

In [ ]:
# 대화 순서 저장(확인용)
def get_talking_sequence(x):
    pattern = '[a-zA-Z\'\s\-,\._가-힣]+:'
    talking_sequence = []
    for talk in x:
        person = re.search(pattern, talk).group(0)
        person = person[:-1]
        talking_sequence.append(person)
        
    return talking_sequence

In [ ]:
# 특정 특수문자 제거
pattern = r"[^a-zA-Z0-9\s!$%&*_+-=~'\"\|:\\.,/?]"
df_train['dialogue'] = df_train['dialogue'].apply(lambda x: remove_extra_spc(x, pattern, ""))

# 개행문자를 제외한 공백문자 제거
df_train['dialogue'] = df_train['dialogue'].apply(lambda x: re.sub(r"[\r\t]", '', x))

# 발화별 리스트화
df_train['dialogue_list'] = df_train['dialogue'].apply(lambda x: x.split("\n"))

# 빈 리스트 제거
df_train['dialogue_list'] = df_train['dialogue_list'].apply(remove_empty)

# 발화별 좌우공백 제거
df_train['dialogue_list'] = df_train['dialogue_list'].apply(lambda x: [item.strip() for item in x])

# 마침표 찍기
df_train['dialogue_list'] = df_train['dialogue_list'].apply(lambda x: [add_fullstop(item) for item in x])

# 연속된 발화자 이어붙히기
df_train['dialogue_list'] = df_train['dialogue_list'].apply(continuous_talking_paltten)

# 예외 데이터 삭제
df_train['dialogue2'] = df_train['dialogue_list'].apply(lambda x: "\n".join(x))
df_train = df_train[~df_train['dialogue2'].str.contains('#Error_Text#')].reset_index(drop=True)

# 발화자 순서 저장하기
df_train['talking_sequence'] = df_train['dialogue_list'].apply(get_talking_sequence)

In [ ]:
df_train

## 3 Solar API

### 3.1 필요한 라이브러리 설치 및 Import

In [ ]:
# !pip install httpx==0.23.2
# !pip install openai==1.2.0

In [ ]:
from openai import OpenAI
import time

### 3.2 API 호출 함수 정의

In [ ]:
with open("solar_api.txt", "r") as f:
    solar_api = f.read()

In [ ]:
def translate_enko(text_en, temperature=0, print_result=False):
    client = OpenAI(
        api_key=solar_api, # <=== [API키 입력]
        base_url="https://api.upstage.ai/v1/solar"
    )

    stream = client.chat.completions.create(
        model="solar-1-mini-translate-enko", # 번역 모델 사용
        messages=[
        {
            "role": "user",
            "content": text_en  # 번역할 텍스트 전달
        }
        ],
        temperature=temperature,
        stream=True,
    )

    trans_str = []
    for chunk in stream:
        test_var = chunk
        if chunk.choices[0].delta.content is not None:
            trans_str.append(chunk.choices[0].delta.content)
            # print(chunk.choices[0].delta.content, end="")
    
    # 출력값 반환
    trans_str = "".join(trans_str)
    # print(trans_str)
    
    if print_result:
        print(f"### Source_text\n{text_en} \n\n### Target_text\n{trans_str}")
    
    return trans_str

In [ ]:
df_train.loc[10, 'dialogue2']

In [ ]:
translate_enko(df_train.loc[10, 'dialogue2'], print_result=True)

### 3.3 API 호출 자동화 및 저장

In [ ]:
bundle_num = 5  # 파일을 저장할 단위(중간에 오류가 나거나 커널이 종료되면 비용만 나가고 데이터가 소멸될 것 대비)
data_point = 10900 // bundle_num    # 시작점(오류 등의 이유로 중지될 경우 다시 시작할 위치)

for i in range(data_point, df_train.shape[0] // bundle_num + 1):
    print(i*bundle_num, i*bundle_num+bundle_num-1)
    start = i*bundle_num
    end = i*bundle_num + bundle_num-1
    
    trans_df = pd.DataFrame(columns=['id', 'ko_summary', 'ko_dialogue'])
    
    temp_df = df_train.loc[start:end].copy()
    temp_df['ko_dialogue'] = temp_df['dialogue2'].apply(translate_enko)     # apply 함수를 활용해 번역
    temp_df['ko_summary'] = temp_df['summary'].apply(translate_enko)
    
    trans_df = temp_df[['id', 'ko_summary', 'ko_dialogue']]
    
    # 번역후 파일 저장
    trans_df.to_csv(f"../data/ko_data/train{start}.csv", index=False)
    
    # Solar API가 Beta 버전이라 그런지 한번에 너무 많은 호출을 하면 Too many request로 오류를 반환하기 때문에 1분을 기다려줌
    print("time sleep")
    time.sleep(60)

### 3.4 번역한 데이터 합치기

In [ ]:
# 폴더 내 모든 csv 파일 경로 가져오기
def get_csv_files(folder_path):
    files = glob.glob(os.path.join(folder_path, '*.csv'))
    return files

In [ ]:
# 폴더 내의 모든 csv 파일 불러와 concat
csv_list= get_csv_files("../data/ko_data")
new_train = pd.DataFrame(columns=['id', 'ko_summary', 'ko_dialogue'])
for csv in csv_list:
    temp_df = pd.read_csv(csv)
    new_train = pd.concat([new_train, temp_df], axis=0)

# 원본 데이터셋과 인덱스 순서 맞춰주기
new_train.dropna(inplace=True)
new_train['id'] = new_train['id'].astype(str)
new_train['id'] = new_train['id'].str.replace(".0", "")

df_train['id'] = df_train['id'].astype(str)
df_train['index'] = df_train.index
temp_df = df_train[['id', 'index', 'dialogue2', 'summary', 'talking_sequence']]

new_train = new_train.merge(temp_df, how='left', on='id')
new_train = new_train.sort_values(by='index').reset_index(drop=True)
del new_train['index']

In [ ]:
new_train.head(1)

## 4 Translated Data Process

In [ ]:
new_train['ko_dialogue'] = new_train['ko_dialogue'].str.strip()
new_train['ko_dialogue_list'] = new_train['ko_dialogue'].str.split("\n")

### 4.1 번역된 요약문 중 구어체로 번역된 요약문 제거

In [ ]:
ko_summary_list = new_train['ko_summary'].tolist()
matched_idxs = []
pattern = r"입니다\.$|입니다$|습니다\.$|습니다$|니다\.$|니다$|이다\.$|이다$|있다\.$|있다$|느낀다\.$|느낀다$|졌다$|졌다\.$|렸다\.$|렸다$|났다\.$|났다$|한다\.$|한다$|않다$|않다\.$|했다$|했다\.$|진다\.$|진다$|랐다\.$|랐다"


for idx, summary in enumerate(ko_summary_list):
    matched = re.search(pattern, summary)
    if matched:
        matched_idxs.append(idx)

print(len(matched_idxs))

new_train = new_train.loc[matched_idxs]
new_train = new_train.reset_index(drop=True)

### 4.2 번역문 중 발화자가 제대로 표시되지 않는 혹은 비어있는 값 제거

In [ ]:
def get_talking_sequence2(x):
    pattern = '[a-zA-Z\'\s\-,\._가-힣]+:'
    talking_sequence = []
    for talk in x:
        matched = re.search(pattern, talk)
        
        if matched:
            person = matched.group(0)
        else:
            person = "#ErrorMatch#:"
        
        person = person[:-1]
        talking_sequence.append(person)
        
    return talking_sequence

In [ ]:
# 발화자 순서 저장하기
new_train['ko_talking_sequence'] = new_train['ko_dialogue_list'].apply(get_talking_sequence2)

In [ ]:
# 발화자를 추출하는 과정에서 오류가 발생했던 데이터 제거
talk_list_list = new_train['ko_talking_sequence'].tolist()
idx_list = []
for idx, talk_list in enumerate(talk_list_list):
    if "#ErrorMatch#" in talk_list:
        idx_list.append(idx)
        
new_train = new_train.drop(idx_list).reset_index(drop=True)

### 4.3 DialogSum 데이터셋과 비슷하게 발화자 마스킹

In [ ]:
from collections import OrderedDict

# DialogSum 데이터와 비슷하게 사람 마스킹하기
ko_dialogue_list = new_train['ko_dialogue_list'].tolist()   # 대화문 리스트
ko_talking_sequence = new_train['ko_talking_sequence'].tolist()     # 발화 순서
ko_summary = new_train['ko_summary'].tolist() # 요약문 리스트

new_dialogue_list = []  # 새롭게 생성할 대화문
new_summary_list = []   # 새롭게 생성할 요약문

for i in range(0, len(ko_dialogue_list)):
    temp_dialogue_list = []
    
    person_list = list(OrderedDict.fromkeys(ko_talking_sequence[i]))    # 순서를 유지한체 중복 제거
    person_dict = {}    # 발화자 별 마스킹값 할당
    for idx, person in enumerate(person_list):
        person_dict[person] = f"#Person{idx+1}#"
    
    # print(ko_dialogue_list[i])
    # print(ko_talking_sequence[i])
    
    # person_dict에 할당된 값에 따라서 발화자 순서에 따라서 마스킹 값으로 대체
    for j in range(0, len(ko_dialogue_list[i])):
        pattern = f"^{ko_talking_sequence[i][j]}:"
        new_str = re.sub(pattern, person_dict[ko_talking_sequence[i][j]]+":", ko_dialogue_list[i][j], count=1)
        temp_dialogue_list.append(new_str)
    
    # person_dict에 할당된 값에 따라서 요약문에 있는 발화자를 마스킹 값으로 대체
    new_summary = ko_summary[i]
    for k, v in person_dict.items():
        # print(k, v)
        pattern = f"{k}"
        new_summary = re.sub(pattern, v, new_summary)
        
    new_dialogue_list.append(temp_dialogue_list)
    new_summary_list.append(new_summary)

### 4.4 데이터 재구성 및 저장

In [ ]:
new_train['processed_dialogue_list'] = new_dialogue_list
new_train['processed_dialogue'] = new_train['processed_dialogue_list'].apply(lambda x: "\n".join(x))
new_train['processed_summary'] = new_summary_list

In [ ]:
new_dataset = new_train[['id', 'ko_summary', 'ko_dialogue', 'processed_dialogue', 'processed_summary']]

In [ ]:
new_dataset.head(3)

In [ ]:
new_dataset.to_csv("../data/ko_dataset.csv", index=False)